In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Firstly, we will create train and test data sets. 

In [ ]:
import matplotlib.pyplot as plt

train_data = pd.read_csv("/kaggle/input/iyzico-projesi/train.csv")
test_data = pd.read_csv("/kaggle/input/iyzico-projesi/test.csv")

Now, let us view features and targets of train and test data. 

In [ ]:
train_data.head()

In [ ]:
train_data.tail()

In [ ]:
train_data.shape

So, train data has 160000 has data value and 12 features. The last column indicates the target values. The column of `ISFLAUD` is target column.

In [ ]:
test_data.tail()

In [ ]:
test_data.shape

In [ ]:
from pandas import DataFrame
import seaborn as sn

df = DataFrame(train_data,columns=['BASKETINSTALLMENT','BASKETPAYMENTCHANNEL','BASKETPAYMENTSOURCETYPE',
                                  'BASKETISTHREEDS', 'BASKETREGISTERCARD', 'BASKETHASVIRTUALITEM', 
                                  'CARDTYPE', 'CARDASSOCIATION'])

corrMatrix = df.corr()
sn.heatmap(corrMatrix, annot=True)

In the data set, the features of `EMAIL`, `CARDBANKID`, `MERCHANT_ID` have not huge effects on prediction of transactions. Because, we don't interest persons who perform transactions, interest only process of transaction. Therefore, we can drop these columns from the train data set. 

In [ ]:
train_data = train_data.drop(["EMAIL", "MERCHANT_ID", "CARDBANKID"], axis= 1)

Now, we check `NaN` values whether exist or not in the data set. 

In [ ]:
count = 0
for row in range(train_data.shape[0]):
    for column in range(train_data.shape[1]):
        if pd.isnull(train_data.iloc[row, column]) == True:
            count += 1
        else:
            continue

if count == 0:
    print('There is no NaN value in the data set')

Now, we can process the clean data. We will Recurrent Neural Network for predicstion. In the Recurrent Neural Network, the method which we use for prediction is LSTM. 

Before struct the network, let us split data(train.csv) for train and test. We will control the strength of the model based on accuracy and loss. Then, we will use updated weigths with the neural network for prediction of data values in the test(test.csv) data. 

In the data set, some features have '0' categorical values. This situation may bring about some problems for compiling the model. So, we will add 1 to all values of some features. 

In [ ]:
features = train_data.iloc[:, :-1]
target = train_data.iloc[:, -1]

In [ ]:
features.shape

In [ ]:
target.shape

In [ ]:
#from sklearn.preprocessing import MinMaxScaler

#minmax_scaler = MinMaxScaler()
#train_data['BASKETPAYMENTCHANNEL'] = minmax_scaler.fit_transform(train_data['BASKETPAYMENTCHANNEL'])

In [ ]:
#feature_set = ['BASKETINSTALLMENT', 'BASKETPAYMENTCHANNEL', 
#               'BASKETPAYMENTSOURCETYPE', 'BASKETISTHREEDS', 'BASKETREGISTERCARD', 
#               'BASKETHASVIRTUALITEM', 'CARDTYPE', 'CARDASSOCIATION']

#for feature_index in features:
#    train_data[feature_index] += 1

In [ ]:
import random

from sklearn.model_selection import train_test_split
train_features, test_features, train_target, test_target = train_test_split(features, target, test_size = 0.33, random_state = random.randrange(0,100))

In [ ]:
count = 0

for index in range(len(test_target)):
    if target[index] == 1:
        count += 1
        
print(count)

In [ ]:
train_features.shape

We need to expand dimension of the train and test features in order to construct suitable network for the data. 

In [ ]:
train_features = np.expand_dims(train_features, axis = 2)
test_features = np.expand_dims(test_features, axis = 2)

In [ ]:
train_features.shape

In [ ]:
import keras 

train_target = keras.utils.to_categorical(train_target)
test_target = keras.utils.to_categorical(test_target)

In [ ]:
train_target.shape

Now, let us create the network. 

In [ ]:
from keras.layers import Dense, Flatten, Dropout, BatchNormalization, Conv1D, MaxPooling1D, LSTM
from keras.models import Sequential 
from keras.regularizers import l1, l2, l1_l2

It is input layer of the network.

In [ ]:
model = Sequential()

There are four hidden layers in the network. `input_shape` indicates the shape of previous layer. In the first hidden layer, `input_shape` must be assigned as dimension of input layer. The `ReLu` function is used as non-linear activation function in all hidden layers. 

In [ ]:
#model.add(Conv1D(64, 2, input_shape = (26, 1), activation = 'relu', kernel_regularizer='l1_l2'))

#model.add(LSTM(64, return_sequences=True, input_shape=(train_features.shape[1], 1), 
#               activation='relu', kernel_regularizer = 'l2'))

model.add(BatchNormalization())
model.add(Dense(850, input_shape=(train_features.shape[1], 1), activation = 'relu', kernel_regularizer='l2'))
model.add(Dropout(0.5))
#model.add(BatchNormalization())
model.add(Dense(900, activation = 'tanh'))
model.add(Dropout(0.5))
model.add(Dense(850, activation = 'relu'))

model.add(Flatten())
#model.add(LSTM(50, return_sequences=True, activation='relu'))
#model.add(LSTM(50, return_sequences=True, activation='tanh'))
#model.add(LSTM(50, return_sequences=False, activation='tanh'))

The final stage of constructing the architecture of the network is setting output layer. 

In [ ]:
#model.add(Dropout(0.5))
#model.add(BatchNormalization())
model.add(Dense(2, activation = 'sigmoid'))

We used `l2 regularization` at output layer and one hidden layer to reduce effect of overfitting. Because, a gap between train and test curves has occured without `l2 regularization`. The gap was reduced with `l2 regularization`. 

We constructed the network. Now, we must compile with optimizer and loss functions 

In [ ]:
opt_param = keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(loss='binary_crossentropy', optimizer = opt_param, metrics=['accuracy'])

Default value of the learning rate is reduced to 0.000001 because the loss and the accuracy values did not change during epochs. This means that the model could not learn anything from data. 

At the final stage, we are fitting with paramaters of `batch_size` and `epoch`. 

In [ ]:
ann = model.fit(train_features, train_target, batch_size=1024, epochs=5, validation_data=(test_features, test_target))

In [ ]:
#Plot the Loss Curves
plt.figure(figsize=[8,6])
train_loss, = plt.plot(ann.history['loss'],'r',linewidth=3.0)
test_loss, = plt.plot(ann.history['val_loss'],'b',linewidth=3.0)
plt.legend([train_loss, test_loss], ['Training Loss', 'Test Loss'],fontsize=12)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)
 
#Plot the Accuracy Curves
plt.figure(figsize=[8,6])
train_accuracy, = plt.plot(ann.history['accuracy'],'r',linewidth=3.0)
test_accuracy, = plt.plot(ann.history['val_accuracy'],'b',linewidth=3.0)
plt.legend([train_accuracy, test_accuracy], ['Training Accuracy', 'Test Accuracy'],fontsize=12)

plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

At the end of 40 epochs, the validation accuracy is around 0.95 and the validation loss is 0.66. The curves of train and test are close to each other and there is no considerable gap between two curves in both accuracy and loss values. Therefore, we can say that there is no overfitting or underfitting. This model can be used for predicting. 

In [ ]:
test_data.head()

In [ ]:
test_data = test_data.drop(["EMAIL", "MERCHANT_ID", "CARDBANKID","ID"], axis=1)

In [ ]:
test_data = np.expand_dims(test_data, axis = 2)

In [ ]:
test_data.shape

Test data is ready for predicting. Let us predict target values with the network model which we created. 

In [ ]:
prediction = model.predict(test_data)

In [ ]:
prediction

We need to classes for `ISFRAUD`. So, we use `predict_classes` function belongs to the model. 

In [ ]:
prediction_classes = model.predict_classes(test_data)

In [ ]:
predictions = list(prediction_classes)

In [ ]:
if 0 in predictions:
    print('yes')

We will create a data frame, then write this data frame to csv file. 

In [ ]:
result_df = pd.DataFrame(columns = ['ID', 'ISFRAUD'])

In [ ]:
transaction_id = []

for index in range(0, len(predictions)):
    transaction_id.append(index)

In [ ]:
result_df['ID'] = transaction_id
result_df['ISFRAUD'] = predictions

In [ ]:
result_df.head()

In [ ]:
result_df.tail()

Finally, the last submission file is created.

In [ ]:
result_df.to_csv('submissions15.csv', index = False)